In [1]:
from datasets import load_dataset

raw_datasets = load_dataset("conll2003",trust_remote_code=True)

c:\Users\priks\anaconda3\envs\t\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 14041
    })
    validation: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 3250
    })
    test: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 3453
    })
})

In [3]:
raw_datasets["train"][0]["tokens"], raw_datasets["train"][0]["ner_tags"], raw_datasets["train"][0]["pos_tags"], raw_datasets["train"][0]["chunk_tags"]

(['EU', 'rejects', 'German', 'call', 'to', 'boycott', 'British', 'lamb', '.'],
 [3, 0, 7, 0, 0, 0, 7, 0, 0],
 [22, 42, 16, 21, 35, 37, 16, 21, 7],
 [11, 21, 11, 12, 21, 22, 11, 12, 0])

In [4]:
ner_feature = raw_datasets["train"].features["ner_tags"]
ner_feature

Sequence(feature=ClassLabel(names=['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-MISC', 'I-MISC'], id=None), length=-1, id=None)

In [5]:
label_names = ner_feature.feature.names
label_names

['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-MISC', 'I-MISC']

In [6]:
words = raw_datasets["train"][0]["tokens"]
labels = raw_datasets["train"][0]["ner_tags"]
line1 = ""
line2 = ""
for word, label in zip(words, labels):
    full_label = label_names[label]
    max_length = max(len(word), len(full_label))
    line1 += word + " " * (max_length - len(word) + 1)
    line2 += full_label + " " * (max_length - len(full_label) + 1)

print(line1)
print(line2)

EU    rejects German call to boycott British lamb . 
B-ORG O       B-MISC O    O  O       B-MISC  O    O 


In [7]:
words = raw_datasets["train"][4]["tokens"]
labels = raw_datasets["train"][4]["ner_tags"]
line1 = ""
line2 = ""
for word, label in zip(words, labels):
    full_label = label_names[label]
    max_length = max(len(word), len(full_label))
    line1 += word + " " * (max_length - len(word) + 1)
    line2 += full_label + " " * (max_length - len(full_label) + 1)

print(line1)
print(line2)

Germany 's representative to the European Union 's veterinary committee Werner Zwingmann said on Wednesday consumers should buy sheepmeat from countries other than Britain until the scientific advice was clearer . 
B-LOC   O  O              O  O   B-ORG    I-ORG O  O          O         B-PER  I-PER     O    O  O         O         O      O   O         O    O         O     O    B-LOC   O     O   O          O      O   O       O 


In [8]:
from transformers import AutoTokenizer

model_checkpoint = "bert-base-cased"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [9]:
tokenizer.is_fast

True

In [10]:
inputs = tokenizer(raw_datasets["train"][0]["tokens"], is_split_into_words=True)
inputs.tokens()

['[CLS]',
 'EU',
 'rejects',
 'German',
 'call',
 'to',
 'boycott',
 'British',
 'la',
 '##mb',
 '.',
 '[SEP]']

In [11]:
inputs.word_ids()

[None, 0, 1, 2, 3, 4, 5, 6, 7, 7, 8, None]

In [12]:
def align_labels_with_tokens(labels, word_ids):
    new_labels = []
    current_word = None
    for word_id in word_ids:
        if word_id != current_word:
            # Start of a new word!
            current_word = word_id
            label = -100 if word_id is None else labels[word_id]
            new_labels.append(label)
        elif word_id is None:
            # Special token
            new_labels.append(-100)
        else:
            # Same word as previous token
            label = labels[word_id]
            # If the label is B-XXX we change it to I-XXX
            if label % 2 == 1:
                label += 1
            new_labels.append(label)

    return new_labels

In [13]:
labels = raw_datasets["train"][0]["ner_tags"]
word_ids = inputs.word_ids()
print(labels)
print(align_labels_with_tokens(labels, word_ids))

[3, 0, 7, 0, 0, 0, 7, 0, 0]
[-100, 3, 0, 7, 0, 0, 0, 7, 0, 0, 0, -100]


In [14]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(
        examples["tokens"], truncation=True, is_split_into_words=True
    )
    all_labels = examples["ner_tags"]
    new_labels = []
    for i, labels in enumerate(all_labels):
        word_ids = tokenized_inputs.word_ids(i)
        new_labels.append(align_labels_with_tokens(labels, word_ids))

    tokenized_inputs["labels"] = new_labels
    return tokenized_inputs

In [15]:
tokenized_datasets = raw_datasets.map(
    tokenize_and_align_labels,
    batched=True,
    remove_columns=raw_datasets["train"].column_names,
)

In [16]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

In [17]:
batch = data_collator([tokenized_datasets["train"][i] for i in range(2)])
batch["labels"]

tensor([[-100,    3,    0,    7,    0,    0,    0,    7,    0,    0,    0, -100],
        [-100,    1,    2, -100, -100, -100, -100, -100, -100, -100, -100, -100]])

In [18]:
for i in range(2):
    print(tokenized_datasets["train"][i]["labels"])

[-100, 3, 0, 7, 0, 0, 0, 7, 0, 0, 0, -100]
[-100, 1, 2, -100]


In [19]:
!pip install seqeval

In [20]:
import evaluate

metric = evaluate.load("seqeval")

In [21]:
labels = raw_datasets["train"][0]["ner_tags"]
labels = [label_names[i] for i in labels]
labels

['B-ORG', 'O', 'B-MISC', 'O', 'O', 'O', 'B-MISC', 'O', 'O']

In [22]:
predictions = labels.copy()
predictions[2] = "O"
metric.compute(predictions=[predictions], references=[labels])

{'MISC': {'precision': 1.0,
  'recall': 0.5,
  'f1': 0.6666666666666666,
  'number': 2},
 'ORG': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 1},
 'overall_precision': 1.0,
 'overall_recall': 0.6666666666666666,
 'overall_f1': 0.8,
 'overall_accuracy': 0.8888888888888888}

In [23]:
import numpy as np


def compute_metrics(eval_preds):
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)

    # Remove ignored index (special tokens) and convert to labels
    true_labels = [[label_names[l] for l in label if l != -100] for label in labels]
    true_predictions = [
        [label_names[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    all_metrics = metric.compute(predictions=true_predictions, references=true_labels)

    # Extract overall metrics
    overall_metrics = {
        "precision": all_metrics["overall_precision"],
        "recall": all_metrics["overall_recall"],
        "f1": all_metrics["overall_f1"],
        "accuracy": all_metrics["overall_accuracy"],
    }

    # Extract per-label metrics
    per_label_metrics = {
        label: {
            "precision": metrics["precision"],
            "recall": metrics["recall"],
            "f1": metrics["f1"],
            "number": metrics["number"],
        }
        for label, metrics in all_metrics.items() if not label.startswith("overall")
    }

    return {
        "overall": overall_metrics,
        "per_label": per_label_metrics,
    }

In [24]:
id2label = {i: label for i, label in enumerate(label_names)}
label2id = {v: k for k, v in id2label.items()}

id2label, label2id 

({0: 'O',
  1: 'B-PER',
  2: 'I-PER',
  3: 'B-ORG',
  4: 'I-ORG',
  5: 'B-LOC',
  6: 'I-LOC',
  7: 'B-MISC',
  8: 'I-MISC'},
 {'O': 0,
  'B-PER': 1,
  'I-PER': 2,
  'B-ORG': 3,
  'I-ORG': 4,
  'B-LOC': 5,
  'I-LOC': 6,
  'B-MISC': 7,
  'I-MISC': 8})

In [25]:
from transformers import AutoModelForTokenClassification

model = AutoModelForTokenClassification.from_pretrained(
    model_checkpoint,
    id2label=id2label,
    label2id=label2id,
)

Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [26]:
model.config.num_labels

9

In [27]:
from transformers import TrainingArguments
from transformers import Trainer

args = TrainingArguments(
    "bert-finetuned-ner-checkpoint",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    num_train_epochs=20,
    weight_decay=0.01,

)


trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=tokenizer,
)


c:\Users\priks\anaconda3\envs\t\lib\site-packages\transformers\training_args.py:1545: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [28]:
trainer.train()

  0%|          | 0/35120 [00:00<?, ?it/s]c:\Users\priks\anaconda3\envs\t\lib\site-packages\transformers\models\bert\modeling_bert.py:440: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:455.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(
  1%|▏         | 505/35120 [00:21<24:22, 23.67it/s]

{'loss': 0.2659, 'grad_norm': 3.4286012649536133, 'learning_rate': 1.9715261958997724e-05, 'epoch': 0.28}


  3%|▎         | 1002/35120 [01:12<59:11,  9.61it/s] 

{'loss': 0.1081, 'grad_norm': 0.7319045066833496, 'learning_rate': 1.9430523917995446e-05, 'epoch': 0.57}


  4%|▍         | 1502/35120 [02:04<56:36,  9.90it/s]  

...

100%|██████████| 35120/35120 [59:54<00:00, 17.48it/s]Trainer is attempting to log a value of "{'precision': 0.9443891102257637, 'recall': 0.9574217435207001, 'f1': 0.9508607721878657, 'accuracy': 0.9872549596750456}" of type <class 'dict'> for key "eval/overall" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'LOC': {'precision': 0.9727371864776445, 'recall': 0.9711486118671747, 'f1': 0.9719422500681013, 'number': 1837}, 'MISC': {'precision': 0.8746113989637305, 'recall': 0.9154013015184381, 'f1': 0.8945416004239534, 'number': 922}, 'ORG': {'precision': 0.9336810730253353, 'recall': 0.9343773303504848, 'f1': 0.9340290719344019, 'number': 1341}, 'PER': {'precision': 0.960169941582581, 'recall': 0.9815418023887079, 'f1': 0.970738255033557, 'number': 1842}}" of type <class 'dict'> for key "eval/per_label" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so w

{'eval_loss': 0.12258101999759674, 'eval_overall': {'precision': 0.9443891102257637, 'recall': 0.9574217435207001, 'f1': 0.9508607721878657, 'accuracy': 0.9872549596750456}, 'eval_per_label': {'LOC': {'precision': 0.9727371864776445, 'recall': 0.9711486118671747, 'f1': 0.9719422500681013, 'number': 1837}, 'MISC': {'precision': 0.8746113989637305, 'recall': 0.9154013015184381, 'f1': 0.8945416004239534, 'number': 922}, 'ORG': {'precision': 0.9336810730253353, 'recall': 0.9343773303504848, 'f1': 0.9340290719344019, 'number': 1341}, 'PER': {'precision': 0.960169941582581, 'recall': 0.9815418023887079, 'f1': 0.970738255033557, 'number': 1842}}, 'eval_runtime': 6.7405, 'eval_samples_per_second': 482.16, 'eval_steps_per_second': 60.381, 'epoch': 20.0}


100%|██████████| 35120/35120 [1:00:04<00:00,  9.74it/s]

{'train_runtime': 3604.2086, 'train_samples_per_second': 77.914, 'train_steps_per_second': 9.744, 'train_loss': 0.013757273165407275, 'epoch': 20.0}


TrainOutput(global_step=35120, training_loss=0.013757273165407275, metrics={'train_runtime': 3604.2086, 'train_samples_per_second': 77.914, 'train_steps_per_second': 9.744, 'total_flos': 6145780901050062.0, 'train_loss': 0.013757273165407275, 'epoch': 20.0})

In [29]:
# Saving the Model and Tokenizer

import json
model_output_dir = "bert-finetuned-ner"

trainer.save_model(model_output_dir)

tokenizer.save_pretrained(model_output_dir)

('bert-finetuned-ner\\tokenizer_config.json',
 'bert-finetuned-ner\\special_tokens_map.json',
 'bert-finetuned-ner\\vocab.txt',
 'bert-finetuned-ner\\added_tokens.json',
 'bert-finetuned-ner\\tokenizer.json')

In [30]:
import os

# Process log_history for structured saving
log_history = trainer.state.log_history

# Extract overall and per-label metrics from evaluation logs
metrics = []
for log in log_history:
    if "eval_loss" in log:  # Check if it's an evaluation log
        metrics_entry = {
            "epoch": log.get("epoch"),
            "step": log.get("step"),
            "eval_loss": log.get("eval_loss"),
            "overall": log.get("eval_overall", {}),
            "per_label": log.get("eval_per_label", {}),
            "runtime": {
                "eval_runtime": log.get("eval_runtime"),
                "samples_per_second": log.get("eval_samples_per_second"),
                "steps_per_second": log.get("eval_steps_per_second"),
            },
        }
        metrics.append(metrics_entry)

# Save overall and per-label metrics to metrics.json
metrics_output_file = os.path.join(model_output_dir, "metrics.json")
with open(metrics_output_file, "w") as f:
    json.dump(metrics, f, indent=4)

# Save the entire log_history to log_history.json
log_history_output_file = os.path.join(model_output_dir, "log_history.json")
with open(log_history_output_file, "w") as f:
    json.dump(log_history, f, indent=4)

print(f"Metrics saved to {metrics_output_file}")
print(f"Log history saved to {log_history_output_file}")

Metrics saved to bert-finetuned-ner\metrics.json
Log history saved to bert-finetuned-ner\log_history.json


In [32]:
import torch
from transformers import TrainingArguments

# Load the training arguments from the bin file
training_args = torch.load(f"{model_output_dir}/training_args.bin")

# Convert to dictionary to view contents
training_args_dict = training_args.to_dict()

# Print the training arguments
for key, value in training_args_dict.items():
    print(f"{key}: {value}")



output_dir: bert-finetuned-ner-checkpoint
overwrite_output_dir: False
do_train: False
do_eval: True
do_predict: False
eval_strategy: epoch
prediction_loss_only: False
per_device_train_batch_size: 8
per_device_eval_batch_size: 8
per_gpu_train_batch_size: None
per_gpu_eval_batch_size: None
gradient_accumulation_steps: 1
eval_accumulation_steps: None
eval_delay: 0
torch_empty_cache_steps: None
learning_rate: 2e-05
weight_decay: 0.01
adam_beta1: 0.9
adam_beta2: 0.999
adam_epsilon: 1e-08
max_grad_norm: 1.0
num_train_epochs: 20
max_steps: -1
lr_scheduler_type: linear
lr_scheduler_kwargs: {}
warmup_ratio: 0.0
warmup_steps: 0
log_level: passive
log_level_replica: warning
log_on_each_node: True
logging_dir: bert-finetuned-ner-checkpoint\runs\Dec01_00-17-24_Prism
logging_strategy: steps
logging_first_step: False
logging_steps: 500
logging_nan_inf_filter: True
save_strategy: epoch
save_steps: 500
save_total_limit: None
save_safetensors: True
save_on_each_node: False
save_only_model: False
restore

In [34]:
# Use a pipeline as a high-level helper
from transformers import pipeline

pipe = pipeline("token-classification", model=model_output_dir)

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


In [35]:
result = pipe('My name is Clara and I live in Berkeley, California.')

In [36]:
result

[{'entity': 'B-PER',
  'score': 0.99972135,
  'index': 4,
  'word': 'Clara',
  'start': 11,
  'end': 16},
 {'entity': 'B-LOC',
  'score': 0.99998116,
  'index': 9,
  'word': 'Berkeley',
  'start': 31,
  'end': 39},
 {'entity': 'B-LOC',
  'score': 0.9999862,
  'index': 11,
  'word': 'California',
  'start': 41,
  'end': 51}]

In [37]:
result = pipe("Bill Gates was the CEO of Microsoft until 2000, when he stepped down and was succeeded by Steve Ballmer12. Gates remained chairman of the board of directors and became chief software architect2. Microsoft's current CEO is Satya Nadella, who took over in 20141.")
result

[{'entity': 'B-PER',
  'score': 0.99998915,
  'index': 1,
  'word': 'Bill',
  'start': 0,
  'end': 4},
 {'entity': 'I-PER',
  'score': 0.9999895,
  'index': 2,
  'word': 'Gates',
  'start': 5,
  'end': 10},
 {'entity': 'B-ORG',
  'score': 0.9999784,
  'index': 7,
  'word': 'Microsoft',
  'start': 26,
  'end': 35},
 {'entity': 'B-PER',
  'score': 0.9999889,
  'index': 19,
  'word': 'Steve',
  'start': 90,
  'end': 95},
 {'entity': 'I-PER',
  'score': 0.9999908,
  'index': 20,
  'word': 'Ball',
  'start': 96,
  'end': 100},
 {'entity': 'I-PER',
  'score': 0.9999912,
  'index': 21,
  'word': '##mer',
  'start': 100,
  'end': 103},
 {'entity': 'I-PER',
  'score': 0.99998987,
  'index': 22,
  'word': '##12',
  'start': 103,
  'end': 105},
 {'entity': 'B-PER',
  'score': 0.9999858,
  'index': 24,
  'word': 'Gates',
  'start': 107,
  'end': 112},
 {'entity': 'B-ORG',
  'score': 0.9999821,
  'index': 39,
  'word': 'Microsoft',
  'start': 195,
  'end': 204},
 {'entity': 'B-PER',
  'score': 0.99

In [38]:
import os
import json
import numpy as np
from transformers import AutoModelForTokenClassification, AutoTokenizer, DataCollatorForTokenClassification
from torch.utils.data import DataLoader
import torch
import evaluate
from tqdm.auto import tqdm


test_metrics_file = os.path.join(model_output_dir, "test_metrics.json")

# Load the model and tokenizer
model = AutoModelForTokenClassification.from_pretrained(model_output_dir)
tokenizer = AutoTokenizer.from_pretrained(model_output_dir)

data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)
test_dataloader = DataLoader(
    tokenized_datasets["test"],  # Replace with your test dataset
    collate_fn=data_collator,
    batch_size=8
)

# Load the seqeval metric
metric = evaluate.load("seqeval")

# Move model to device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)


# Testing loop
model.eval()
progress_bar = tqdm(test_dataloader)
all_logits = []
all_labels = []

max_seq_len = 0

# First pass: Determine the maximum sequence length
for batch in test_dataloader:
    max_seq_len = max(max_seq_len, batch["input_ids"].size(1))

# Second pass: Collect padded logits and labels
for batch in progress_bar:
    with torch.no_grad():
        # Move batch to device
        batch = {key: val.to(device) for key, val in batch.items()}
        
        # Forward pass
        outputs = model(**batch)
        
        # Pad logits and labels to the maximum sequence length
        logits_padded = np.pad(
            outputs.logits.cpu().numpy(),
            ((0, 0), (0, max_seq_len - outputs.logits.size(1)), (0, 0)),
            mode="constant",
        )
        labels_padded = np.pad(
            batch["labels"].cpu().numpy(),
            ((0, 0), (0, max_seq_len - batch["labels"].size(1))),
            mode="constant",
            constant_values=-100,
        )
        all_logits.append(logits_padded)
        all_labels.append(labels_padded)

# Concatenate all logits and labels
all_logits = np.concatenate(all_logits, axis=0)
all_labels = np.concatenate(all_labels, axis=0)

# Compute test metrics
test_metrics = compute_metrics((all_logits, all_labels))

# Define helper function to handle JSON serialization
def convert_to_serializable(obj):
    if isinstance(obj, np.integer):
        return int(obj)
    elif isinstance(obj, np.floating):
        return float(obj)
    elif isinstance(obj, np.ndarray):
        return obj.tolist()
    else:
        raise TypeError(f"Object of type {type(obj).__name__} is not JSON serializable")


# Save test metrics to a JSON file
with open(test_metrics_file, "w") as f:
    json.dump(test_metrics, f, indent=4, default=convert_to_serializable)

# Display test metrics
print("Test metrics saved to", test_metrics_file)
print(json.dumps(test_metrics, indent=4, default=convert_to_serializable))



100%|██████████| 432/432 [00:03<00:00, 124.74it/s]


Test metrics saved to bert-finetuned-ner\test_metrics.json
{
    "overall": {
        "precision": 0.899030135088327,
        "recall": 0.9190864022662889,
        "f1": 0.9089476448958151,
        "accuracy": 0.9727391626353193
    },
    "per_label": {
        "LOC": {
            "precision": 0.9286570743405276,
            "recall": 0.9286570743405276,
            "f1": 0.9286570743405276,
            "number": 1668
        },
        "MISC": {
            "precision": 0.7555266579973993,
            "recall": 0.8276353276353277,
            "f1": 0.7899388171312032,
            "number": 702
        },
        "ORG": {
            "precision": 0.8832159624413145,
            "recall": 0.9060806742925949,
            "f1": 0.8945022288261516,
            "number": 1661
        },
        "PER": {
            "precision": 0.9528475199020208,
            "recall": 0.9622758194186766,
            "f1": 0.9575384615384616,
            "number": 1617
        }
    }
}


In [10]:
from transformers import pipeline

# Replace with your specific model checkpoint
model_checkpoint = "bert-finetuned-ner"
token_classifier = pipeline(
    "token-classification", 
    model=model_checkpoint, 
    aggregation_strategy="simple"
)

# Example usage
result = token_classifier("My name is Sylvain and I work at Hugging Face in Brooklyn.")
print(result)

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


[{'entity_group': 'PER', 'score': 0.9999881, 'word': 'Sylvain', 'start': 11, 'end': 18}, {'entity_group': 'ORG', 'score': 0.99961376, 'word': 'Hugging Face', 'start': 33, 'end': 45}, {'entity_group': 'LOC', 'score': 0.99989843, 'word': 'Brooklyn', 'start': 49, 'end': 57}]
